In [1]:
import torch
print(torch.cuda.is_available())  # should be True
print(torch.cuda.get_device_name(0))  # GPU name

True
NVIDIA GeForce RTX 3090


In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model


2025-09-30 14:51:27.563632: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-30 14:51:27.563658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-30 14:51:27.564433: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-30 14:51:27.568251: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-30 14:51:28.115364: W tensorflow/compiler/tf2

In [3]:
df = pd.read_csv("hf://datasets/Gaborandi/breast_cancer_pubmed_abstracts/breast_cancer.csv")
df

,pubmed_id,title,abstract
0,36800818,Evaluating the effect of nurses supportive and...,Endometriosis is one of the womens aggressive ...
1,36800748,18F-FDG PET-CT in Cutaneous Metastases in Brea...,Cutaneous and subcutaneous metastases are an e...
2,36800716,Efficacy of pre - operative axillary ultrasono...,"To determine false negative rate, negative pre..."
3,36800710,Is complete pathological response truly a comp...,To check if complete pathological response in ...
4,36800640,Experiences of a Multiethnic Cohort of Patient...,Financial reimbursement programs (FRPs) offset...
...,...,...,...
9508,36194067,The Lumbar Artery Perforator Flap in Breast Re...,The lumbar artery perforator flap is a valuabl...
9509,36194066,Sensory Restoration in Abdominally Based Free ...,Neurotization in breast reconstruction can be ...
9510,31915335,StatPearls,"Breast lumps or masses are very common, partic..."
9511,26389406,PDQ Cancer Information Summaries,This PDQ cancer information summary has curren...


In [4]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Shape of training set: {train_df.shape}")
print(f"Shape of validation set: {validation_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of training set: (7610, 3)
Shape of validation set: (951, 3)
Shape of test set: (952, 3)


In [6]:
from huggingface_hub import login
import os

hf_token = os.environ.get("HF_TOKEN")
login(hf_token)

In [7]:
"""
# === Qwen3-32B 4-bit GPU setup and generation ===

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# --- Step 1: BitsAndBytes 4-bit config ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# --- Step 2: Load tokenizer ---
model_dir = "Qwen/Qwen3-32B"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

# --- Step 3: Load model on GPU(s) ---
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto",       # Automatically distributes layers across GPUs
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    offload_folder="offload",  # Optional: offload layers to CPU if GPU memory is low
    offload_state_dict=True
)

# Disable caching to save GPU memory
model.config.use_cache = False
model.config.pretraining_tp = 1

# --- Step 4: Check GPU devices ---
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Model device map:", model.hf_device_map)

# --- Step 5: Generate text ---
prompt = "Once upon a time in a futuristic city,"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move inputs to GPU

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.95,
        temperature=0.8
    )

# Decode and print
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("=== Generated Text ===")
print(generated_text)
"""

'\n# === Qwen3-32B 4-bit GPU setup and generation ===\n\nfrom transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig\nimport torch\n\n# --- Step 1: BitsAndBytes 4-bit config ---\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_use_double_quant=False,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_compute_dtype=torch.bfloat16,\n)\n\n# --- Step 2: Load tokenizer ---\nmodel_dir = "Qwen/Qwen3-32B"\ntokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)\n\n# --- Step 3: Load model on GPU(s) ---\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_dir,\n    quantization_config=bnb_config,\n    device_map="auto",       # Automatically distributes layers across GPUs\n    torch_dtype=torch.bfloat16,\n    trust_remote_code=True,\n    offload_folder="offload",  # Optional: offload layers to CPU if GPU memory is low\n    offload_state_dict=True\n)\n\n# Disable caching to save GPU memory\nmodel.config.use_cache = False\nmodel.config.pretr

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# --- Step 1: 4-bit config for VRAM efficiency ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# --- Step 2: Load tokenizer ---
model_dir = "Qwen/Qwen2-0.5B"  # Choose the 0.5B model
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

# --- Step 3: Load model across multiple GPUs ---
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto",       # Automatically shard across all available GPUs
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

# Disable caching to save VRAM
model.config.use_cache = False

# --- Step 4: Determine first device for inputs ---
first_device = list(model.hf_device_map.values())[0]

# --- Step 5: Generate text ---
prompt = "Once upon a time in a futuristic city,"
inputs = tokenizer(prompt, return_tensors="pt").to(first_device)  # Place on first shard

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.95,
        temperature=0.8
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("=== Generated Text ===")
print(generated_text)

# --- Step 6: Optional info ---
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Model device map:", model.hf_device_map)


`torch_dtype` is deprecated! Use `dtype` instead!
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


=== Generated Text ===
Once upon a time in a futuristic city, the only way to get around was the bike. Every vehicle must be painted black and black is a very popular color. As a result, the city was nicknamed "Black City." Black cars and bikes were used in a strange way to solve problems. One day, the city's people were attacked by a band of aliens, and the city was put under a state of emergency.
After the aliens have destroyed the city, the city's inhabitants and their leaders want to build a new city. The city
CUDA available: True
Number of GPUs: 2
Model device map: {'': 1}


In [10]:
def combine_qa(row):
    return f"Question: {row['title']}\nAnswer: {row['abstract']}\n"

train_texts = train_df.apply(combine_qa, axis=1).tolist()
val_texts = validation_df.apply(combine_qa, axis=1).tolist()


In [11]:
model_name = "Qwen/Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=512)

train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)



In [12]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",        # Multi-GPU
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False


In [13]:
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.input_ids = encodings["input_ids"]
        self.attention_mask = encodings["attention_mask"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "attention_mask": torch.tensor(self.attention_mask[idx]),
            "labels": torch.tensor(self.input_ids[idx])
        }

train_dataset = QADataset(train_encodings)
val_dataset = QADataset(val_encodings)


In [14]:
"""
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",  # Automatically shard across GPUs
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False

"""

'\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_use_double_quant=False,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_compute_dtype=torch.bfloat16,\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_name,\n    quantization_config=bnb_config,\n    device_map="auto",  # Automatically shard across GPUs\n    torch_dtype=torch.bfloat16,\n    trust_remote_code=True\n)\n\nmodel.config.use_cache = False\n\n'

In [15]:
device = torch.device("cuda:0")  # your GPU

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": device},  # pin model to single GPU
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False


In [16]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # adjust if needed
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [17]:
training_args = TrainingArguments(
    output_dir="./qwen2-0.5B-finetuned-breastcancer",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    fp16=True,
    gradient_accumulation_steps=4,
    save_total_limit=2
)


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


[codecarbon WARNING @ 14:52:30] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 14:52:30] [setup] RAM Tracking...
[codecarbon INFO @ 14:52:30] [setup] CPU Tracking...
[codecarbon WARNING @ 14:52:31] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 14:52:31] CPU Model on constant consumption mode: AMD Ryzen 7 5800X 8-Core Processor
[codecarbon WARNING @ 14:52:31] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 14:52:31] [setup] GPU Tracking...
[codecarbon INFO @ 14:52:31] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 14:52:31] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: cpu_load
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 14:

In [19]:
trainer.train()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bobbyhieubui (bobbyhieubui-university-of-wisconsin-madison). Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.590800
20,1.782500
30,0.849500
40,0.761800
50,0.750500
60,0.738700
70,0.713500
80,0.715800
90,0.760000
100,0.743400


[codecarbon INFO @ 14:52:52] Energy consumed for RAM : 0.000086 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 14:52:53] Delta energy consumed for CPU with cpu_load : 0.000045 kWh, power : 10.5246044547 W
[codecarbon INFO @ 14:52:53] Energy consumed for All CPU : 0.000045 kWh
[codecarbon INFO @ 14:52:53] Energy consumed for all GPUs : 0.001296 kWh. Total GPU Power : 291.3482739124505 W
[codecarbon INFO @ 14:52:53] 0.001427 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:53:07] Energy consumed for RAM : 0.000167 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 14:53:08] Delta energy consumed for CPU with cpu_load : 0.000042 kWh, power : 10.524781603218754 W
[codecarbon INFO @ 14:53:08] Energy consumed for All CPU : 0.000088 kWh
[codecarbon INFO @ 14:53:08] Energy consumed for all GPUs : 0.002585 kWh. Total GPU Power : 309.54667849815144 W
[codecarbon INFO @ 14:53:08] 0.002840 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:53:22] Energy consumed for RAM : 0.0

TrainOutput(global_step=1428, training_loss=0.7412545624233428, metrics={'train_runtime': 774.1957, 'train_samples_per_second': 29.489, 'train_steps_per_second': 1.844, 'total_flos': 2.513865965174784e+16, 'train_loss': 0.7412545624233428, 'epoch': 3.0})

In [20]:
model.save_pretrained("./qwen2-0.5B-lora-finetuned-breastcancer")
print("✅ Fine-tuning complete. LoRA weights saved.")


✅ Fine-tuning complete. LoRA weights saved.


In [21]:
"""
import evaluate

# Load BLEU metric
bleu = evaluate.load("bleu")

predictions = []
references = []

for idx in range(len(val_dataset)):
    input_ids = val_dataset[idx]["input_ids"].unsqueeze(0).to(model.device)
    attention_mask = val_dataset[idx]["attention_mask"].unsqueeze(0).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=512)

    pred_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    ref_text = tokenizer.decode(val_dataset[idx]["labels"], skip_special_tokens=True)

    predictions.append(pred_text)       # <-- full string
    references.append([ref_text])       # <-- list of strings

# Compute BLEU
bleu_score = bleu.compute(predictions=predictions, references=references)
print("BLEU score:", bleu_score["bleu"])
"""

'\nimport evaluate\n\n# Load BLEU metric\nbleu = evaluate.load("bleu")\n\npredictions = []\nreferences = []\n\nfor idx in range(len(val_dataset)):\n    input_ids = val_dataset[idx]["input_ids"].unsqueeze(0).to(model.device)\n    attention_mask = val_dataset[idx]["attention_mask"].unsqueeze(0).to(model.device)\n\n    with torch.no_grad():\n        output_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=512)\n\n    pred_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)\n    ref_text = tokenizer.decode(val_dataset[idx]["labels"], skip_special_tokens=True)\n\n    predictions.append(pred_text)       # <-- full string\n    references.append([ref_text])       # <-- list of strings\n\n# Compute BLEU\nbleu_score = bleu.compute(predictions=predictions, references=references)\nprint("BLEU score:", bleu_score["bleu"])\n'

In [25]:
from torch.utils.data import DataLoader
import evaluate

# DataLoader to batch the validation set
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
bleu = evaluate.load("bleu")
predictions, references = [], []

for batch in val_loader:
    input_ids = batch["input_ids"].to(model.device)
    attention_mask = batch["attention_mask"].to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=128,
            do_sample=False
        )

    # Decode predictions
    batch_preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    # Decode references (fix -100s first)
    batch_refs = []
    for labels in batch["labels"]:
        labels = [tok if tok != -100 else tokenizer.pad_token_id for tok in labels]
        batch_refs.append([tokenizer.decode(labels, skip_special_tokens=True)])

    predictions.extend(batch_preds)
    references.extend(batch_refs)

# Compute BLEU
bleu_score = bleu.compute(predictions=predictions, references=references)
print("BLEU score:", bleu_score["bleu"])


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_tok

BLEU score: 0.9771004697738882
